# MMC Implementation

## Correct Implementation

In [1]:
import numpy as np
#import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import warnings
import skmob
from tqdm import tqdm
import sys
sys.path.append('../')
from src.geo_utils import (get_clusters_from_tdf,
                            assign_tdf_points_to_clusters,
                            get_mmc_transitions,
                            get_stationary_vector)


In [2]:
warnings.filterwarnings('ignore')

In [99]:
def get_distance_bw_clusters(cluster_1, cluster_2):
    '''Get Distance Between Clusters
    
    Parameters
    ----------
        cluster_1 (pd.Dataframe): df with columns ['cluster', 'lat', 'lng', 'sta_vector']
        cluster_2 (pd.Dataframe): df with columns ['cluster', 'lat', 'lng', 'sta_vector']
    
    Returns
    -------
        distance (float): Distance between clusters (one-way)
    '''
    valid_distances = []
    valid_clusters = []
    for i, row_i in cluster_1.iterrows():    
        coord_i = (row_i['lat'], row_i['lng'])

        clusters, distances = [], []    
        for j, row_j in cluster_2.iterrows():
            cluster_j = row_j['cluster']
            coord_j = (row_j['lat'], row_j['lng'])

            distance_ij = skmob.utils.utils.distance(coord_i, coord_j)    
            clusters.append(cluster_j)
            distances.append(distance_ij)

        distances = np.array(distances)

        idx_min = distances.argmin()
        min_dist = distances.min()
        cluster_min = clusters[idx_min]

        valid_distances.append(min_dist)
        valid_clusters.append(cluster_min)

    cluster_est = cluster_1.copy()
    cluster_est['cluster_other'] = valid_clusters
    cluster_est['distance_other'] = valid_distances
    distance = (cluster_est['distance_other'] * cluster_est['sta_vector']).sum()
    return distance

def get_mean_distance_bw_clusters(cluster_1, cluster_2):
    '''Get Mean Distance Between Clusters
    
    Get the distances of cluster_1 to cluster_2 and viceversa and returns the average.

    Parameters
    ----------
        cluster_1 (pd.Dataframe): df with columns ['cluster', 'lat', 'lng', 'sta_vector']
        cluster_2 (pd.Dataframe): df with columns ['cluster', 'lat', 'lng', 'sta_vector']
    
    Returns
    -------
        mean_distance (float): Distance between clusters (two-way)
    '''
    d1 = get_distance_bw_clusters(cluster_1, cluster_2)
    d2 = get_distance_bw_clusters(cluster_2, cluster_1)
    mean_distance = (d1+d2)/2
    return mean_distance

In [8]:
def get_mmc_clusters_stavectors(geo):
    '''Get MMC Clusters and Stationary Vectors
    
    Parameters
    ----------
        geo (pd.Dataframe): Dataframe of 1 user_id. Must contain only the columns: ['user', 'hour', 'lat', 'lng']
    
    Returns
    -------
        clusters (pd.Dataframe): Dataframe with the clusters and the stationary value
        m (folium.folium.Map): Map generated with the clusters on it.
    '''
    # TDF Definition
    trgeo = skmob.TrajDataFrame(
        geo, 
        datetime='hour',
        user_id='user'
    )

    # Cluster Generation
    clusters, m = get_clusters_from_tdf(
        trgeo,
        verbose=True,
        max_speed_kmh= 0.01,
        detect_stops=False,
        compress=False ,
        minutes_for_a_stop=2,
        spatial_radius_km=0.2,
        spatial_radius_compress_km=.2,
        cluster_radius_km=0.5,
        min_samples=2
    )

    # Cluster Assignation
    trgeo_cl, distances = assign_tdf_points_to_clusters(
        tdf=trgeo, 
        clusters=clusters
    )

    # Generation of Transit Dataframe
    transit_df = get_mmc_transitions(trgeo_cl)

    # Generation of Transit Matrix
    transit_matrix = pd.crosstab(transit_df['cluster'], 
                                 transit_df['cluster_next'],
                                 normalize='index').values

    # Stationary Vector Assignation
    clusters['sta_vector'] = get_stationary_vector(transit_matrix)

    return clusters, m

In [24]:
#url_usr82 = 'https://raw.githubusercontent.com/bitmapup/mmc3/master/data/geolife_82.csv'
#geo82 = pd.read_csv(url_usr82, header=None)

geo_columns = ['user', 'hour', 'lat', 'lng']
geo = pd.read_csv('../data/geo82.csv.zip')
geo.columns = geo_columns

## Reading for 10 users

In [29]:
# Reading 10 frames
data = pd.read_csv('../data/geo_000_009.csv.zip')

data['hour'] = data['date'] + ' ' + data['time']
data['user'] = data['user_id'].map(int)

data = data[geo_columns]

In [43]:
# Extraction of clusters 
geo_clusters = {}
geo_clusters_img = {}
users = sorted(data['user'].unique())
for user in tqdm(users):
    try:
        geo = data[data['user']==user].reset_index(drop=True).copy()

        clusters, m = get_mmc_clusters_stavectors(geo)

        geo_clusters[user] = clusters
        geo_clusters_img[user] = m
    except:
        pass

  0%|          | 0/2 [00:00<?, ?it/s]

INFO: Noise Filtering applied
INFO: Clusters generated
(173870, 4) (211, 4) (211, 4) (211, 4) (211, 5)
INFO: 2 clusters generated.



  0%|          | 0/4 [00:00<?, ?it/s]

INFO: Noise Filtering applied
INFO: Clusters generated
(108607, 4) (46, 4) (46, 4) (46, 4) (46, 5)
INFO: 4 clusters generated.



  0%|          | 0/6 [00:00<?, ?it/s]

...


  0%|          | 0/1 [00:00<?, ?it/s]

INFO: Noise Filtering applied
INFO: Clusters generated
(77910, 4) (41, 4) (41, 4) (41, 4) (41, 5)
INFO: 1 clusters generated.



  0%|          | 0/2 [00:00<?, ?it/s]

INFO: Noise Filtering applied
INFO: Clusters generated
(84616, 4) (41, 4) (41, 4) (41, 4) (41, 5)
INFO: 2 clusters generated.



100%|██████████| 10/10 [03:20<00:00, 20.06s/it]


In [104]:
# TODO: Corregir la función de vector estacionario. Hay casos en los que falla

In [44]:
geo_clusters_img

{0: <folium.folium.Map at 0x7fe4c7698950>,
 3: <folium.folium.Map at 0x7fe4c78b3390>,
 4: <folium.folium.Map at 0x7fe4f954f990>}

## Distance Estimation

In [101]:
cluster_0 = geo_clusters[0]
cluster_1 = geo_clusters[3]
cluster_2 = geo_clusters[4]

In [52]:
cluster_1

,cluster,lat,lng,sta_vector
0,0,39.999859,116.326749,0.474359
1,1,39.987315,116.309833,0.384615
2,2,39.995477,116.288653,0.141026


In [51]:
cluster_2

,cluster,lat,lng,sta_vector
0,0,39.999944,116.326035,0.312925
1,1,39.939100,116.348471,0.136054
2,2,39.966866,116.378085,0.054422
3,3,39.986601,116.381269,0.108844
4,4,39.974776,116.331752,0.217687
5,5,39.974543,116.415310,0.013605
6,6,39.987192,116.398921,0.061224
7,7,39.987036,116.389978,0.095238


In [100]:
get_mean_distance_bw_clusters(cluster_1=cluster_1, cluster_2=cluster_2)

2.245555609708128

In [102]:
get_mean_distance_bw_clusters(cluster_1=cluster_0, cluster_2=cluster_2)

1.9047901696538245